In [1]:
import gc
import os
import pickle
import sys
import time
import uuid
from typing import Final, Optional

import cv2 as OpenCV
import numpy as np
from matplotlib import pyplot as plt
from numpy.linalg import norm
from scipy.cluster.vq import kmeans, vq
from scipy.spatial import Delaunay

In [2]:
class Image:
    def __init__(self, img_id, rgb_image, gray_image, keypoints, descriptors, path):
        self.img_id: int = int(img_id)
        self.unique_id: uuid = uuid.uuid4()
        self.rgb_image: Image = rgb_image
        self.gray_image: Image = gray_image
        self.keypoints: list[OpenCV.KeyPoint] = keypoints
        self.descriptors: np.ndarray = descriptors
        self.path: str = path

    @property
    def length(self):
        return f"{len(self.keypoints)}" if len(self.keypoints) == len(self.descriptors) else f"{len(self.keypoints)}, {len(self.descriptors)}"
    
    def draw_sift_features(self):
        image_with_sift = OpenCV.drawKeypoints(self.rgb_image, self.keypoints, None, flags=OpenCV.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        plt.imshow(image_with_sift)
        plt.title("Image with SIFT Features")
        plt.axis('off')
        plt.show()

    def display_rgb_image(self, title: Optional[str] = None):
        image = self.rgb_image
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        plt.axis('off')
        plt.show()

    def display_gray_image(self, title: Optional[str] = None):
        image = self.gray_image
        plt.gray()
        plt.imshow(image)
        if title is not None:
            plt.title(title)
        plt.axes('off')
        plt.show()

        
    def __repr__(self):
        return f"Image({self.img_id})"
    
    def __str__(self):
        return self.__repr__()
    
    def __eq__(self, other):
        return self.unique_id == other.unique_id
    
    def __hash__(self):
        return hash(self.img_id)
    
    def __getstate__(self):
        state = self.__dict__.copy()
        state['keypoints'] = [tuple(k.pt) + (k.size, k.angle, k.response, k.octave, k.class_id) for k in self.keypoints]
        return state
    
    def __setstate__(self, state):
        state['keypoints'] = [OpenCV.KeyPoint(x, y, size, angle, response, octave, class_id) for x, y, size, angle, response, octave, class_id in state['keypoints']]
        self.__dict__ = state

class FeatureMatches:
    def __init__(self, image_one: Image, image_two: Image, matches: list[OpenCV.DMatch]):
        self.image_one: Image = image_one
        self.image_two: Image = image_two
        self.matches: list[OpenCV.DMatch] = matches

    def draw_matches(self, output_filename: str) -> None:
        combined_image = OpenCV.hconcat([
            self.image_one.rgb_image,
            self.image_two.rgb_image
        ])

        for match in self.matches:
            x1, y1 = self.image_one.keypoints[match.queryIdx].pt
            x2, y2 = self.image_two.keypoints[match.trainIdx].pt
            # Draw a line connecting the matched keypoints
            OpenCV.line(
                combined_image, 
                (int(x1), int(y1)), 
                (int(x2) + self.image_one.rgb_image.shape[1], int(y2)), 
                (0, 255, 0), 
                1
            )

        OpenCV.imwrite(output_filename, combined_image)

    def __repr__(self):
        return f"FeatureMatches({self.image_one}, {self.image_two} ---> {len(self.matches)})"

    def __getstate__(self):
        state = self.__dict__.copy()
        state['matches'] = [
            {'queryIdx': m.queryIdx, 'trainIdx': m.trainIdx, 'distance': m.distance} for m in self.matches
        ]
        return state
    
    def __setstate__(self, state):
        state['matches'] = [
            OpenCV.DMatch(match['queryIdx'], match['trainIdx'], match['distance']) for match in state['matches']
        ]
        self.__dict__ = state
    
class Images:
    def __init__(self, images: list[Image], image_set_name: str):
        self.id = uuid.uuid4()
        self.images: list[Image] = images
        self.image_set_name: str = image_set_name
        self.feature_matches: list[FeatureMatches] = []
        self.similar_images: dict[list[Image]] = {}
        self.num_clusters: int = 50

    def save_feature_matches(self):
        for match in self.feature_matches:
            match.draw_matches(f"data/snow-man/output/feature-match/{match.image_one.img_id}_{match.image_two.img_id}.jpg")

    def __len__(self):
        return len(self.images)
    
    def display_similar_images(self, key):
        print(f"cluster {key}")
        print("-----------------------------------------------------")
        for value in self.similar_images[key]:
            print(value)
            rgb_image = OpenCV.cvtColor(OpenCV.imread(value.path), OpenCV.COLOR_BGR2RGB)
            plt.imshow(rgb_image)
            plt.title(value.path)
            plt.axis('off')
            plt.show()

    def save_similar_images(self):
        for cluster in self.similar_images.keys():
            if not os.path.exists(f"data/{self.image_set_name}/output/image-match/{cluster}"):
                os.makedirs(f"data/{self.image_set_name}/output/image-match/{cluster}")
            for value in self.similar_images[cluster]:
                OpenCV.imwrite(f"data/{self.image_set_name}/output/image-match/{cluster}/{value.img_id}.jpg", value.rgb_image)

    def __getitem__(self, key: int) -> Image:
        for image in self.images:
            if image.img_id == key:
                return image
        raise KeyError(f'Image with img_id {key} not found.')
    


In [30]:
def feature_matching(
        img_one_descriptors: np.ndarray, 
        img_two_descriptors: np.ndarray,
        knn: bool = False,
        **kwargs
    ) -> list[OpenCV.DMatch]:
    if knn:
        matcher = OpenCV.BFMatcher()
        return matcher.knnMatch(img_one_descriptors, img_two_descriptors, k=1)
    else:
        matcher = OpenCV.BFMatcher(crossCheck=True)
        return matcher.match(img_one_descriptors, img_two_descriptors)


def apply_ransac(matches, keypoints1, keypoints2, threshold = 3.0, **kwargs):
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    _, mask = OpenCV.findHomography(src_pts, dst_pts, OpenCV.RANSAC, threshold)
    matches_mask = mask.ravel().tolist()
    return [m for m, keep in zip(matches, matches_mask) if keep]

def data_feature_matching(images: Images) -> None:
    import itertools
    for key, values in images.similar_images.items():
        for image, matched_image in itertools.combinations(values, 2):
            feature_matching_output = feature_matching(image.descriptors, matched_image.descriptors)
            ransac_output = apply_ransac(feature_matching_output, image.keypoints, matched_image.keypoints, threshold=150)
            images.feature_matches.append(FeatureMatches(image, matched_image, ransac_output))

In [3]:
def load_images_bak(images_file_path: str) -> Images:
    """ Load images from a file """
    with open(images_file_path, "rb") as file:
        images = pickle.load(file)
    return images

In [4]:
images = load_images_bak(f"data/snow-man/bak/feature-matching-output.pkl")

In [17]:
def keep_top_k_matches(matches, k):
    sorted_matches = sorted(matches, key=lambda x: x.distance)
    return sorted_matches[:k]

In [34]:
image_1 = images[34]
image_2 = images[35]

print(image_1.rgb_image.shape, image_2.rgb_image.shape)

feature_matching_output = feature_matching(image_1.descriptors, image_2.descriptors)
print(len(feature_matching_output))
matches_before = FeatureMatches(image_1, image_2, feature_matching_output)
matches_before.draw_matches("test_feature_match.jpg")

# knn_feature_matching_output = feature_matching(image_1.descriptors, image_2.descriptors, True)
# print(len(knn_feature_matching_output))
# matches_before = FeatureMatches(image_1, image_2, knn_feature_matching_output)
# matches_before.draw_matches("test_knn_feature_match.jpg")

ransac_output = apply_ransac(feature_matching_output, image_1.keypoints, image_2.keypoints, threshold=50)
matches = FeatureMatches(image_1, image_2, ransac_output)
matches.draw_matches("test_feature_match_ransac.jpg")
print(len(ransac_output))

# ransac_output = apply_ransac(knn_feature_matching_output, image_1.keypoints, image_2.keypoints, threshold=150)
# matches = FeatureMatches(image_1, image_2, ransac_output)
# matches.draw_matches("test_knn_feature_match_ransac.jpg")
# print(len(ransac_output))


# feature_matching_output = keep_top_k_matches(feature_matching_output, 1000)
# matches_after = FeatureMatches(image_1, image_2, feature_matching_output)
# matches_after.draw_matches("test_feature_match_sort.jpg")
# print(len(feature_matching_output))




(4624, 2604, 3) (4624, 2604, 3)
7557
105


In [ ]:
@timeit
def compute_deep_features(images: list[Image], **kwargs) -> None:
    """Compute deep features for each image in the list of images using ResNet50 model.
    Modifies each image in the list of images by adding its deep features as an attribute.
    
    Args:
    - images: List of images to compute deep features for.

    Returns:
    - None.
    """
    image_set_name = kwargs['image_set_name']
    model = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    for img in images.images:
        img_data = keras_image.img_to_array(img.color_image)
        img_data = np.expand_dims(img_data, axis=0)
        img_data = preprocess_input(img_data)

        deep_features = model.predict(img_data)
        img.deep_features = deep_features
        log_to_file(f"data/{image_set_name}/logs/tune.log", f"Img({img.img_id}, {img.path}) has deep features of shape {img.deep_features.shape}.")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

@timeit
def data_feature_matching(images: Images, similarity_threshold: float = 0.8, **kwargs) -> None:
    """ Match features between images using cosine similarity
    Args:
        images: Images object containing images with their deep features
        similarity_threshold: Threshold to consider two images as matching (default: 0.8)
    Returns:
        None
    """
    import itertools
    image_set_name = kwargs['image_set_name']
    for key, values in images.similar_images.items():
        log_to_file(f"data/{image_set_name}/logs/tune.log", f"Started Feature Match for cluster number {key}:")
        for image, matched_image in itertools.combinations(values, 2):
            similarity = cosine_similarity(image.deep_features, matched_image.deep_features)[0][0]
            
            if similarity > similarity_threshold:
                images.feature_matches.append(FeatureMatches(image, matched_image, similarity))
                log_to_file(f"data/{image_set_name}/logs/tune.log", f"({image.img_id}, {matched_image.img_id}) with similarity {similarity}.")